In [2]:
import os
import cv2
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn import preprocessing, model_selection

In [3]:
data_path = './data/'
if not os.path.exists(data_path):
    os.makedirs(data_path, exist_ok=True)
    with zipfile.ZipFile('archive.zip', 'r') as zip_f:
        zip_f.extractall(data_path)

In [5]:
size=128
data = []
label = []
folder_path = f"{data_path}/flowers"
for folder in os.listdir(folder_path):
    for file in os.listdir(os.path.join(folder_path, folder)):
        if file.endswith('jpg'):
            label.append(folder)
            img = cv2.imread(os.path.join(folder_path, folder, file))
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            im = cv2.resize(img_rgb, (size, size))
            data.append(im)
        else:
            continue

In [7]:
set(label)

{'daisy', 'dandelion', 'rose', 'sunflower', 'tulip'}

In [10]:
np.array(data).shape

(4317, 128, 128, 3)

In [11]:
len(label)

4317

In [12]:
data = np.array(data)
label = np.array(label)

In [13]:
encoder = preprocessing.LabelEncoder()
y = encoder.fit_transform(label)
y = tf.keras.utils.to_categorical(y, 5)
x = data/255

In [15]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, test_size=0.2, random_state=32)

In [18]:
x_train.shape, x_test.shape

((3453, 128, 128, 3), (864, 128, 128, 3))

In [26]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', input_shape=(size, size, 3)))
model.add(tf.keras.layers.MaxPool2D(pool_size=2))
          
model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'))
model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'))
model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=2))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(rate=0.5))
model.add(tf.keras.layers.Dense(5, activation='softmax'))

In [27]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=20,
                                                         zoom_range=0.20,
                                                          width_shift_range=0.3,
                                                         height_shift_range=0.3,
                                                         horizontal_flip=True,
                                                         vertical_flip=True)

In [28]:
datagen.fit(x_train)
tf.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop

In [29]:
model.compile(optimizer = tf.keras.optimizers.Adam(0.0001),
             loss = 'categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
history_1 = model.fit_generator(datagen.flow(x_train, y_train,batch_size=32),
                                             epochs=10,
                                             validation_data=(x_test, y_test),
                                             verbose=0)

C:\Users\Lenovo2\AppData\Local\Temp\ipykernel_20024\3546935401.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history_1 = model.fit_generator(datagen.flow(x_train, y_train,batch_size=32),
